In [2]:
# Getting Started
!uv pip install pynetlogo

# Set the path to your NetLogo installation
# Note! This is OS and machine dependent.
# Perhaps someone has an NL docker image out there?
# See https://pynetlogo.readthedocs.io/en/latest/_docs/pynetlogo.html#pynetlogo.core.NetLogoLink
netlogo_path = '/Applications/NetLogo 6.3.0'  # mac osx


error: No virtual environment found; run `uv venv` to create an environment, or pass `--system` to install into a non-virtual environment


# Module 2 Assignment: Schelling Segregation Model

*CAS 520, Peter Dresslar*

For this assignment we will use [pyNetLogo](https://pynetlogo.readthedocs.io/) to capture and report simulation outputs. An excellent and relevant example appears [here](https://pynetlogo.readthedocs.io/en/latest/_docs/introduction.html)

## Step 1: First Analysis

### Experiment 1: Set density to 80% and similarity wanted to 30% and run the simulation several times.

### Experiment 2: Keeping density at 80%, change the similarity threshold to 90%.

### Experiment 3: While the model is still running from step 2, slowly move the similarity slider to the left and note when the model behaviour changes and segregation emerges.

### Experiment 4: Rerun the simulation a few times with 80% density and similarity threshold at the tipping point you discovered.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pynetlogo

netlogo = pynetlogo.NetLogoLink(
    gui=True,
    netlogo_home=netlogo_path
)
model = "./Segregation.nlogo" # (copied into this directory!)

def find_breakpoint():
    """
    This is tricky! We start with the given density and similarity wanted.
    We know from manual testing that there is a breakpoint at which point the model 
    will reduce to 0% unhappy. We also can observe that this "collapse" occurs within
    1000 ticks, and that the go procedure will halt the model at that point.
    
    We can run a simple binary "search" to find this point,
    by running the model with similarity-wanted values between 1.0 and 90.0.

    If the model stops within 1000 ticks, we know that the breakpoint is higher than the 
    searched similarity-wanted value. If the model runs the full 1000 steps, we know that
    the breakpoint is lower than the searched similarity-wanted value.

    Since we are setting values programatically, we have the luxury of looking beyond the integer values of similarity-wanted.


    """
    density = 80.0
    similarity_wanted = 90.0
    upper_bound = similarity_wanted
    lower_bound = 1.0 # not sure how the model would deal with zeroes!
    breakpoint = None
    
    iteration = 0
    while breakpoint is None:
        iteration += 1
        print(f"Iteration {iteration}")
        midpoint = round((upper_bound + lower_bound) / 2.0, 1) # round to 1 decimal place
        netlogo.load_model(model)
        netlogo.command(f"set density {density}")
        netlogo.command(f"set %-similar-wanted {midpoint}")
        netlogo.command("setup")
        
        netlogo.command("repeat 1000 [go]")
        # we should be able to see if the model halted early by checking the step number 

        print(netlogo.report("ticks"))

        if netlogo.report("ticks") < 1000:
            lower_bound = midpoint + 0.1
        else:
            upper_bound = midpoint - 0.1

        if upper_bound - lower_bound < 0.1:
            breakpoint = midpoint
    
    return breakpoint
        
    


def experiment(number_of_runs, density, similar_wanted):
    data = []

    for run in range(number_of_runs):
        netlogo.load_model(model)
        netlogo.command("set density " + density)
        netlogo.command("set similarity-wanted " + similar_wanted)
        netlogo.command("setup")

        netlogo.command("repeat 100 [go]")


print(find_breakpoint())





ModuleNotFoundError: No module named 'pynetlogo'